# Introducción

Durante todo este curso vamos a estar contruyendo un compilador de COOL, paso a paso, introduciendo nuevas características del lenguaje o mejorando la implementación de otras características a medida que vamos descubriendo las técnicas fundamentales de la teoría de lenguajes y la compilación.

El objetivo de esta clase es construir un evaluador de expresiones "a mano", usando los recursos que tenemos hasta el momento. Para ello vamos a comenzar con una versión de COOL muy sencilla, un lenguaje de expresiones aritméticas.

En este lenguaje, que llamaremos `COOL 0.1` o `xCOOL` (la `x` por `expression`), podemos usar expresiones como la siguiente:

`32.4 + 5 * ( e - sin ( 2 * pi ) )`

## COOL 0.1 (xCOOL)

Definiremos a continuación este lenguaje de manera informal:

Un programa en `xCOOL` consta de una secuencia de expresiones. Cada expresión está compuesta por:

- números (con coma flotante de 32 bits), 
- operadores `+ - * / ^` con el orden operacional, 
- paréntesis `(` y `)`, 
- constantes `pi`, `e`, etc., y
- funciones elementales `sin`, `cos`, `tan`, `log`, `sqrt`, etc.

## Evaluando expresiones simples

Comenzaremos construyendo un prototipo bien simple, donde solamente aceptaremos números y operadores. Además, asumiremos que en la expresión hay espacios en blanco entre todos los elementos, de modo que el *lexer* se reduce a dividir por espacios. Luego iremos adicionando elementos más complejos.

El siguiente método devuelve una lista de *tokens*, asumiendo que la expresión solo tiene números y operadores, separados por espacios en blanco.

In [1]:
def tokenize(expr):
    """
    Returns the set of tokens. At this point, simply splits by 
    spaces and converts numbers to `float` instances.
    """
    tokens = []
    
    for token in expr.split():
        # :solution:
        try:
            tokens.append(float(token))
        except:
            tokens.append(token)
        # :final:
        # Insert your code here ...
        pass
        # :end:
    
    return tokens

In [2]:
expr = '3.2 + 6.5 * 5'
tokens = tokenize(expr)

assert tokens == [3.2, '+', 6.5, '*', 5.0]

Una vez que tenemos los *tokens*, solo nos queda evaluar la expresión. El primer algoritmo que usaremos es muy sencillo: buscamos el operador de menor prioridad, y evaluamos recursivamente a ambos lados de la expresión.

In [3]:
# These numbers indicate the priority, the lower number the higher the priority.
priority = {
    '^': 0,
    '*': 1,
    '/': 1,
    '+': 2,
    '-': 2,
}

# These lambda expressions map from operators to actual executable code
operations = {
    '^': lambda x,y: x ** y,
    '*': lambda x,y: x * y,
    '/': lambda x,y: x / y,
    '+': lambda x,y: x + y,
    '-': lambda x,y: x - y,
}

In [4]:
def evaluate(tokens):
    """
    Evaluates an expression recursively.
    """
    if not tokens:
        raise ValueError("Ill-formed expression")
    
    if len(tokens) == 1:
        # Must be a number
        return tokens[0]
    
    last_oper = None
    last_prior = -1
    
    # :solution:
    for i, token in enumerate(tokens):
        if token in priority:
            prior = priority[token]
            
            if prior > last_prior:
                last_oper = i
                last_prior = prior
                
    if last_prior < 0:
        raise ValueError("Ill-formed expression")
    # :final:
    # Insert your code here ...
    # Find the operator with the least priority
    # :end:
        
    left_expr = evaluate(tokens[:last_oper])
    right_expr = evaluate(tokens[last_oper+1:])
    
    operation = operations[tokens[last_oper]]

    return operation(left_expr, right_expr)

In [5]:
value = evaluate(tokens)
assert value == 35.7

## Adicionando paréntesis

Ahora que ya tenemos las funcionalidades básicas, vamos a adicionar los paréntesis. Para ello, sólo nuestro método de evaluación tiene que cambiar, dado que al tokenizador no le deberían importar los paréntesis. En caso de no ser así, es hora de regresar y arreglarlo.

In [6]:
expr = "( 3.2 + ( 6.5 - 2.3 ) ) * 5"
tokens = tokenize(expr)

assert tokens == ['(', 3.2, '+', '(', 6.5, '-', 2.3, ')', ')', '*', 5.0]

Para resolver la prioridad con respecto a los paréntesis, solamente necesitamos adicionar un concepto de *nivel de profundidad*. Cada vez aparece un paréntesis abierto aumentamos el nivel de profundidad, y lo decrementamos al cerrar un paréntesis. Los operadores en un nivel de profundidad mayor siempre tienen prioridad sobre aquellos en un nivel de profundidad menor. A la misma vez estaremos chequeando que la cadena tenga los paréntesis balanceados.

**`(!)`** Cuidado con los paréntesis adicionales en los extremos: `( 2 + 3 )` ...

In [7]:
def evaluate(tokens):
    """
    Evaluates an expression recursively.
    """
    if not tokens:
        raise ValueError("Ill-formed expression")
    
    if len(tokens) == 1:
        # Must be a number
        return tokens[0]
    
    last_oper = None
    last_prior = -1
    last_level = 1e50
    current_level = 0
    
    # :solution:
    for i, token in enumerate(tokens):
        if token == '(':
            current_level += 1
            continue
    
        if token == ')':
            current_level -= 1
            
            if current_level < 0:
                raise ValueError("Ill-formed expression")
                
            continue
            
        if token in priority:
            prior = priority[token]

            if current_level < last_level or (current_level == last_level and prior > last_prior):
                last_oper = i
                last_prior = prior
                last_level = current_level
                
    if last_prior < 0:
        raise ValueError("Ill-formed expression")
    
    if last_level > 0:
        tokens = tokens[last_level:-last_level]
        last_oper -= last_level
    # :final:
    # Insert your code here ...
    # Find the operator with the least priority
    # :end:
        
    left_expr = evaluate(tokens[:last_oper])
    right_expr = evaluate(tokens[last_oper+1:])
    
    operation = operations[tokens[last_oper]]

    return operation(left_expr, right_expr)

In [8]:
assert evaluate(tokens) == 37.0

# Constantes

Para adicionar las constantes, simplemente añadiremos un diccionario con todas las constantes disponibles, que usaremos durante la tokenización.

In [9]:
constants = {
    'pi': 3.14159265359,
    'e': 2.71828182846,
    'phi': 1.61803398875,
}

In [10]:
def tokenize(expr):
    """
    Returns the set of tokens. At this point, simply splits by 
    spaces and converts numbers to `float` instances.
    Replaces constants.
    """
    tokens = []
    
    for token in expr.split():
        # :solution:
        try:
            tokens.append(float(token))
        except:
            tokens.append(constants.get(token, token))
        # :final:
        # Insert your code here ...
        pass
        # :end:
    
    return tokens

In [11]:
expr = '2 * pi'
tokens = tokenize(expr)

assert tokens == [2.0, '*', 3.14159265359]

# Funciones elementales

Para las funciones elementales haremos algo similar a las constantes, pero en vez de a la hora de tokenizar, las reemplazaremos a la hora de evaluar, pues necesitamos evaluar recursivamente los argumentos de la función. Empezaremos por garantizar que nuestro tokenizador que es capaz de reconocer expresiones con funciones elementales de más de un argumento, en caso de no ser así es necesario arreglarlo.

In [12]:
expr = 'log ( 64 , 1 + 3 )'
tokens = tokenize(expr)

assert tokens == ['log', '(', 64.0, ',', 1.0, '+', 3.0, ')']

Adicionaremos entonces un diccionario con todas las funciones elementales que permitiremos.

In [13]:
import math

functions = {
    'sin': lambda x: math.sin(x),
    'cos': lambda x: math.cos(x),
    'tan': lambda x: math.tan(x),
    'log': lambda x,y: math.log(x, y),
    'sqrt': lambda x: math.sqrt(x),
}

Por último, modificaremos el método `evaluate`. Esto lo vamos a hacer en varios pasos, primero crearemos un método auxiliar `evaluate_function`, que asumiremos siempre recibe una sub-expresión que consiste exactamente en un llamado a función. Este método se encargará de evaluar recursivamente los argumentos de la función y luego invocar a la función en sí. Los argumentos están separados por el token `,`. 

Recordemos que cada uno de los argumentos puede a su vez tener sub-expresiones que consistan también en llamados a funciones, por lo es necesario llevar la cuenta del *nivel de profundidad*.

In [14]:
def evaluate_function(expr):
    """
    Receives a tokenized expression that consists exactly in a function
    invocation and recursively evaluates the arguments and then the function itself.
    """
    if len(expr) < 3:
        raise ValueError('Ill-formed expression')

    fun = expr[0]
    
    if not fun in functions:
        raise ValueError('Invalid function: %s' % fun)
        
    if expr[1] != '(' or expr[-1] !=')':
        raise ValueError('Ill-formed expression')
        
    expr = expr[1:-1]
    
    args = []
    current_level = 0
    current_arg = []
    
    # :solution:
    for token in expr[1:]:
        if token == '(':
            current_level += 1
            continue
            
        if token == ')':
            current_level -= 1
            
            if current_level < 0:
                raise ValueError('Ill-formed expression')
                
            continue
            
        if current_level == 0 and token == ',':
            args.append(evaluate(current_arg))
            current_arg = []
            continue
            
        current_arg.append(token)
        
    if current_arg:
        args.append(evaluate(current_arg))
    # :final:
    # Split and evaluate all arguments
    # ...
    # :end:
        
    return functions[fun](*args)

In [15]:
assert evaluate_function(tokens) == 3.0

Ahora que tenemos una forma de evaluar una función, tenemos que modificar nuestro método `evaluate` para usar esta funcionalidad. Dado que este método está tan orientado a expresiones en notación **infija**, y ya se está volviendo bastante largo, vamos a tener que hacer varias modificaciones. 

Comenzaremos por separar la funcionalidad que encuentra el operador de menor precedencia en un método aparte. Este método recibirá una expresión (en forma de lista de tokens) y nos devolverá el índice del token que representa el operador más externo. Este método asumirá que pueden existir paréntesis innecesarios a los lados de la expresión.

**`(!)`** Recordemos que la invocación de una función tiene la máxima prioridad.

In [16]:
def find_top_operator(tokens):
    if not tokens:
        raise ValueError("Ill-formed expression")
    
    last_oper = None
    last_prior = -1
    last_level = 1e50
    current_level = 0
    
    # :solution:
    for i, token in enumerate(tokens):
        prior = None
        
        if token == '(':
            current_level += 1
            continue
    
        if token == ')':
            current_level -= 1
            
            if current_level < 0:
                raise ValueError("Ill-formed expression")
                
            continue
            
        if token in functions:
            prior = 100
        elif token in priority:
            prior = priority[token]
            
        if prior:
            if current_level < last_level or (current_level == last_level and prior > last_prior):
                last_oper = i
                last_prior = prior
                last_level = current_level
                
    if last_prior < 0:
        raise ValueError("Ill-formed expression")
    # :final:
    # Insert your code here ...
    # :end:
    
    return last_oper

In [17]:
expr = '( ( sin ( 2 * pi ) / tan ( 4 ) ) + cos ( pi - 1 ) )'
tokens = tokenize(expr)

assert find_top_operator(tokens) == 15